In [1]:
import pandas as pd 
import ast
import json
from pandas import json_normalize

.
## ==================================================
# Boliga_propertySales_evaluationInfo.csv
## ==================================================
.

## ----- read in the file : 

In [ ]:
# path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\12_evalutationInfos\Boliga_propertySales_evalutationInfos_.csv'
# data_evaluationInfos = pd.read_csv(path, encoding='utf-8',low_memory=False)
# data_evaluationInfos.info()

In [ ]:
# data_evaluationInfos['RAW_dictionary_evaluationInfos'][0]

### --------- Convert "RAW_dictionary_salesInfos" column to dictionary object                

In [ ]:
# def decode_json(x):
#     try:
#         if isinstance(x, str) and x.strip() != "" and not pd.isna(x):
#             return ast.literal_eval(x)
#             # Alternatively, you can use json.loads as follows:
#             # return json.loads(x.replace("'", '"'))
#         else:
#             return x
#     except (ValueError, SyntaxError) as e:
#         print(f"Error decoding JSON: {e}")
#         return None

# # Apply the decoding function to the specified column
# data_evaluationInfos['RAW_dictionary_evaluationInfos'] = data_evaluationInfos['RAW_dictionary_evaluationInfos'].apply(decode_json)

In [ ]:
# data_evaluationInfos['RAW_dictionary_evaluationInfos'][4]

# !!!!!! Data to big for RAM memory !!!!!!!

.

.

# PLAN: 

* Read in the file in chunks of 1000 rows per chunk (so 1038000/10000 = 104 chunks)
* ONLY READ the main ID columns and "RAW_dictionary_evaluationInfos" column
* for each chunk as a dataframe convert column "RAW_dictionary_evaluationInfos" into a dictionary object 
* then expand the column into rows i.e. 1 row = 1 dictionary
* then noramlize all dictionaries into columns
* Finally write result to csv file and move on to next chunk 

In [ ]:
# Specify your file path
file_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\12_evalutationInfos\Boliga_propertySales_evalutationInfos_.csv'

result_path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\12_evalutationInfos\Ready\Boliga_propertySales_evaluationInfo_Ready.csv'

# Define chunk size
chunk_size = 10000

# select only necessary columns :
fields = ['RowID_MAIN_boliga_ROW_ID_unitID', 'RAW_dictionary_evaluationInfos']

# Create an iterator for reading chunks
chunk_iter = pd.read_csv(file_path, chunksize=chunk_size, usecols=fields)

# Initialize a flag to track whether it's the first iteration for the HEADER OF THE RESULTING CSV FILE
first_iteration = True

# count the chunks 
chunk_counter = 0

# Iterate through each chunk
for i, chunk in enumerate(chunk_iter):
    chunk_counter+=1
    print(f'----------> chunk nr. {chunk_counter} out of 104 circa')
    
    # Convert 'RAW_dictionary_evaluationInfos' column to dictionary
    # chunk['RAW_dictionary_evaluationInfos'] = chunk['RAW_dictionary_evaluationInfos'].apply(ast.literal_eval)
    # chunk['RAW_dictionary_evaluationInfos'] = chunk['RAW_dictionary_evaluationInfos'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip() != '' else x)
    print('----> Convert to dictionary object')
    chunk['RAW_dictionary_evaluationInfos'] = chunk['RAW_dictionary_evaluationInfos'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) and x != '' and x !='[]' else x)

    # Explode the columns so that each item in the list gets its own row
    print('----> Explode column')
    chunk = chunk.explode('RAW_dictionary_evaluationInfos').reset_index(drop=True)

    # Normalize the column into columns and drop then drop the column with pop
    print('----> Normalize and join')
    chunk = chunk.join(json_normalize(chunk.pop('RAW_dictionary_evaluationInfos')))

    # ONLY TAKE VALUATIONS BETWEEN 2006 TO 2024
    chunk = chunk[(chunk['evaluationYear'] >= 2006) & (chunk['evaluationYear'] <= 2024)]


    # # Expand the dictionary column into rows
    # expanded_df = pd.json_normalize(exploded_df['RAW_dictionary_evaluationInfos'])
    
    # # Concatenate the expanded dataframe with the original dataframe
    # result_df = pd.concat([chunk, expanded_df], axis=1)
    
    # # Drop the original dictionary column
    # result_df = result_df.drop(columns=['RAW_dictionary_evaluationInfos'])
    
    # Write the result to the existing CSV file (mode='a' appends to the existing file)
    print('----> Write result to CSV')
    chunk.to_csv(result_path, mode='a', index=False, header=first_iteration)

    # Update the flag after the first iteration
    first_iteration = False

    # Release memory by resetting variables and deleting unnecessary objects
    # del exploded_df, expanded_df, result_df

In [4]:
1038900/10000

103.89

## Concurrent futures 

# ----- Rename columns:

In [ ]:
change_column_names = {
    'isSalesValid':'salesInfo_isSalesValid', 
    'handoverCode':'salesInfo_handoverCode', 
    'handoverName':'salesInfo_handoverName', 
    'deedIssueDate':'salesInfo_deedIssueDate', 
    'price':'salesInfo_price',
    'recalculationDate':'salesInfo_recalculationDate', 
    'rebuildYear':'salesInfo_rebuildYear'    
}

data_salesInfo = data_salesInfo.rename(columns=change_column_names)
data_salesInfo.info()

## ------ Save the result 

In [ ]:
sub_columns = [
    'RowID_MAIN_boliga_ROW_ID_unitID', 

    'salesInfo_isSalesValid',   
    'salesInfo_handoverCode', 
    'salesInfo_handoverName', 
    'salesInfo_deedIssueDate', 
    'salesInfo_price',
    'salesInfo_recalculationDate', 
    'salesInfo_rebuildYear'
]

path=r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\11_salesInfos\Ready\Boliga_propertySales_salesInfos_Ready.csv'

data_salesInfo[sub_columns].to_csv(path, encoding='utf-8', index=False)

In [ ]:
data_additionarBuildingInfo.info()